In [1]:
import pandas as pd
import urllib3
import requests
from bs4 import BeautifulSoup
from io import BytesIO
import gzip

In [2]:
df = pd.read_csv("input/raw/cc_golf.csv")
df.head()

,Unnamed: 0,charset,digest,filename,languages,length,mime,mime-detected,offset,status,timestamp,url,urlkey
0,0,UTF-8,HUOE7NY2VG7HAL66CF6G5N5SSNYWSEHG,crawl-data/CC-MAIN-2019-13/segments/1552912202...,eng,25543,text/html,text/html,275210809,200,20190321022742,http://www.espn.com/golf/story/_/id/10012503/s...,"com,espn)/golf/story/_/id/10012503/sony-extend..."
1,1,UTF-8,6Z4VBZ6IFQXUCS7N6IYITFCT3MPWL2SK,crawl-data/CC-MAIN-2019-13/segments/1552912202...,eng,27178,text/html,text/html,280588463,200,20190319212914,http://www.espn.com/golf/story/_/id/10080442/t...,"com,espn)/golf/story/_/id/10080442/tiger-woods..."
2,2,UTF-8,UBD4YLQOW2KLYPKRELWXGEBAFKFZ4I54,crawl-data/CC-MAIN-2019-13/segments/1552912201...,eng,27561,text/html,text/html,278478889,200,20190319074141,http://www.espn.com/golf/story/_/id/10244068/z...,"com,espn)/golf/story/_/id/10244068/zach-johnso..."
3,3,UTF-8,6UCNXIS4Z25R6ZWR6D7RXS6RYIV6AO5N,crawl-data/CC-MAIN-2019-13/segments/1552912201...,eng,28292,text/html,text/html,280156073,200,20190319075048,http://www.espn.com/golf/story/_/id/10249216/w...,"com,espn)/golf/story/_/id/10249216/webb-simpso..."
4,4,UTF-8,P6XF4K3CSSJNLGWYKVMO524TVLBZ4HFZ,crawl-data/CC-MAIN-2019-13/segments/1552912207...,eng,31685,text/html,text/html,278025653,200,20190327024410,http://www.espn.com/golf/story/_/id/10257010/t...,"com,espn)/golf/story/_/id/10257010/tiger-woods..."


In [3]:
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    try:
        resp = requests.get(
            prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

        raw_data = BytesIO(resp.content)
        f = gzip.GzipFile(fileobj=raw_data)

    except OSError as e:
        print(e)
        return ""
        
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""

    if len(data):

        try:
            # data = data.decode("utf-8")
            # print(data)

            [warc, header, response] = data.strip().split(b'\r\n\r\n', 2)
            # print(warc, header)
        except Exception as e:
            print(e)
            return response

    return response

In [4]:
def get_text_from_html(html_content):
    text = ""
    soup = BeautifulSoup(html_content, features="html.parser")

    matches = soup.select(
        "section > article:first-child > div.container > div.article-body > p")

    for item in matches:
        text += item.get_text()+"\n"
    return text

In [5]:
text = ""
for idx, row in df.iterrows():
    if row['status'] == 200:
        html_content = download_page(row)

        print("[*] Retrieved %d bytes for %s" %
              (len(html_content), row['url']))
        if len(html_content) != 0:
            text += get_text_from_html(html_content)

[*] Retrieved 109313 bytes for http://www.espn.com/golf/story/_/id/10012503/sony-extends-sponsorship-pga-tour-hawaii-event-2018
[*] Retrieved 110758 bytes for http://www.espn.com/golf/story/_/id/10080442/tiger-woods-talks-2014-majors-lindsey-vonn
[*] Retrieved 112110 bytes for http://www.espn.com/golf/story/_/id/10244068/zach-johnson-leads-pga-tour-tournament-champions
[*] Retrieved 115796 bytes for http://www.espn.com/golf/story/_/id/10249216/webb-simpson-dustin-johnson-jordan-spieth-tied-lead-kapalua
[*] Retrieved 186125 bytes for http://www.espn.com/golf/story/_/id/10257010/tiger-woods-hits-13-billion-career-earnings-report-says
[*] Retrieved 110730 bytes for http://www.espn.com/golf/story/_/id/10398739/juli-inkster-helps-lpga-return-bay-area-swinging-skirts-lpga-classic
[*] Retrieved 109253 bytes for http://www.espn.com/golf/story/_/id/10554988/olympic-golf-qualifying-date-set-july-11-2016
[*] Retrieved 118988 bytes for http://www.espn.com/golf/story/_/id/10614233/john-senden-punch

[*] Retrieved 118362 bytes for http://www.espn.com/golf/story/_/id/15876975/pga-tour-move-event-miami-trump-doral-resort-mexico-city-2017
[*] Retrieved 114526 bytes for http://www.espn.com/golf/story/_/id/15890076/oregon-edges-texas-first-men-golf-title
[*] Retrieved 118196 bytes for http://www.espn.com/golf/story/_/id/15978028/william-mcgirt-hardscrabble-journey-pays-memorial-tournament
[*] Retrieved 113248 bytes for http://www.espn.com/golf/story/_/id/16152913/britain-ireland-defeat-us-curtis-cup
[*] Retrieved 119258 bytes for http://www.espn.com/golf/story/_/id/16964418/penalty-anna-nordqvist-delivers-us-women-open-brittany-lang
[*] Retrieved 114420 bytes for http://www.espn.com/golf/story/_/id/16984051/jordan-spieth-says-compete-rio-olympics
[*] Retrieved 119392 bytes for http://www.espn.com/golf/story/_/id/17077239/phil-mickelson-shoots-major-record-tying-63-first-round-open
[*] Retrieved 118234 bytes for http://www.espn.com/golf/story/_/id/17212815/nike-announces-exit-golf-equipm

[*] Retrieved 114117 bytes for http://www.espn.com/golf/story/_/id/19536797/golf-looking-presidents-cup-india-anirban-lahiri-gaining-ground
[*] Retrieved 114797 bytes for http://www.espn.com/golf/story/_/id/19672538/canada-brooke-henderson-wins-meijer-lpga-classic
[*] Retrieved 113631 bytes for http://www.espn.com/golf/story/_/id/19727337/so-yeon-ryu-shoots-course-record-61-nw-arkansas-championship
[*] Retrieved 118356 bytes for http://www.espn.com/golf/story/_/id/19865737/lpga-confidential-survey-speaking-golf-inequalities
[*] Retrieved 133980 bytes for http://www.espn.com/golf/story/_/id/19955182/inspiring-greatness-chinese-lpga-golfer-shanshan-feng-does-things-way
[*] Retrieved 118131 bytes for http://www.espn.com/golf/story/_/id/20082582/sung-hyun-park-wins-us-women-open-1st-victory
[*] Retrieved 115824 bytes for http://www.espn.com/golf/story/_/id/20083422/17-year-old-amateur-hye-jin-choi-misses-chance-win-us-women-open
[*] Retrieved 122903 bytes for http://www.espn.com/golf/story

[*] Retrieved 121191 bytes for http://www.espn.com/golf/story/_/id/22726636/tiger-woods-rory-mcilroy-joe-lacava-jj-watt-reggie-jackson-weigh-tiger-big-week
[*] Retrieved 123951 bytes for http://www.espn.com/golf/story/_/id/22726636/tiger-woods-rory-mcilroy-joe-lacava-jj-watt-reggie-jackson-weigh-tiger-big-week
[*] Retrieved 120570 bytes for http://www.espn.com/golf/story/_/id/22726648/valspar-championship-doubt-no-more-tiger-woods-going-win-again
[*] Retrieved 109411 bytes for http://www.espn.com/golf/story/_/id/22727339/vijay-singh-takes-toshiba-classic-1st-individual-win-pga-tour-champions
[*] Retrieved 113597 bytes for http://www.espn.com/golf/story/_/id/22727339/vijay-singh-takes-toshiba-classic-1st-individual-win-pga-tour-champions
[*] Retrieved 114785 bytes for http://www.espn.com/golf/story/_/id/22732387/new-golf-rules-include-height-penalty-drops-relaxed-penalty-rules
[*] Retrieved 116439 bytes for http://www.espn.com/golf/story/_/id/22734879/interest-golf-leaps-tiger-woods-con

[*] Retrieved 123449 bytes for http://www.espn.com/golf/story/_/id/23083247/caddie-confidential-patrick-reed-caddies-explains-was-watch-man-win-green-jacket-2018-masters-tournament
[*] Retrieved 125656 bytes for http://www.espn.com/golf/story/_/id/23083247/caddie-confidential-patrick-reed-caddies-explains-was-watch-man-win-green-jacket-2018-masters-tournament
[*] Retrieved 127789 bytes for http://www.espn.com/golf/story/_/id/23084137/masters-2018-rory-mcilroy-jordan-spieth-two-different-feelings-comes-augusta-national
[*] Retrieved 127794 bytes for http://www.espn.com/golf/story/_/id/23084137/masters-2018-rory-mcilroy-jordan-spieth-two-different-feelings-comes-augusta-national
[*] Retrieved 123623 bytes for http://www.espn.com/golf/story/_/id/23091216/masters-2018-3-miles-away-patrick-reed-family-wept-watched-win
[*] Retrieved 125285 bytes for http://www.espn.com/golf/story/_/id/23091216/masters-2018-3-miles-away-patrick-reed-family-wept-watched-win
[*] Retrieved 125932 bytes for http:

...

[*] Retrieved 122104 bytes for http://www.espn.com/golf/story/_/id/23772463/jordan-spieth-frustrated-impatient-how-year-gone-far
[*] Retrieved 121990 bytes for http://www.espn.com/golf/story/_/id/23772463/jordan-spieth-frustrated-impatient-how-year-gone-far
[*] Retrieved 106610 bytes for http://www.espn.com/golf/story/_/id/23772463/jordan-spieth-frustrated-impatient-how-year-gone-far
[*] Retrieved 116435 bytes for http://www.espn.com/golf/story/_/id/23773360/pga-tour-greenbrier-classic-changes-tournament-name-honor-military
[*] Retrieved 101859 bytes for http://www.espn.com/golf/story/_/id/23773360/pga-tour-greenbrier-classic-changes-tournament-name-honor-military
[*] Retrieved 129701 bytes for http://www.espn.com/golf/story/_/id/23779334/everything-need-know-round-1-2018-us-open
[*] Retrieved 116021 bytes for http://www.espn.com/golf/story/_/id/23779334/everything-need-know-round-1-2018-us-open
[*] Retrieved 122023 bytes for http://www.espn.com/golf/story/_/id/23781966/rory-mcilroy-pl

[*] Retrieved 121916 bytes for http://www.espn.com/golf/story/_/id/24341809/gary-woodland-matches-36-hole-scoring-record-pga-championship
[*] Retrieved 124639 bytes for http://www.espn.com/golf/story/_/id/24359528/how-brooks-koepka-outlasted-stacked-leaderboard-win-pga-championship-bellerive
[*] Retrieved 124155 bytes for http://www.espn.com/golf/story/_/id/24359752/brooks-koepka-holds-tiger-woods-adam-scott-win-pga-championship
[*] Retrieved 129163 bytes for http://www.espn.com/golf/story/_/id/24360463/tiger-woods-always-hated-coming-second-all-bad-year-pga-championship-where-finished-brooks-koepka
[*] Retrieved 125110 bytes for http://www.espn.com/golf/story/_/id/24360666/brooks-koepka-losing-pga-championship-bellerive-anyone-not-even-tiger-woods
[*] Retrieved 124185 bytes for http://www.espn.com/golf/story/_/id/24364687/jim-furyk-commit-tiger-woods-ryder-cup-large-pick
[*] Retrieved 117519 bytes for http://www.espn.com/golf/story/_/id/24370119/daniel-hillier-trevor-phillips-atop-us-

[*] Retrieved 123316 bytes for http://www.espn.com/golf/story/_/id/24951390/justin-thomas-said-decision-team-jordan-spieth-known-ahead-ryder-cup
[*] Retrieved 120119 bytes for http://www.espn.com/golf/story/_/id/24955506/ryder-cup-francesco-molinari-left-unable-tie-shoelaces-record-performance
[*] Retrieved 118534 bytes for http://www.espn.com/golf/story/_/id/24983521/bernhard-langer-runs-away-sas-championship
[*] Retrieved 119050 bytes for http://www.espn.com/golf/story/_/id/24991926/paul-azinger-replace-retiring-johnny-miller-nbc
[*] Retrieved 115485 bytes for http://www.espn.com/golf/story/_/id/24993274/laura-davies-leads-juli-inkster-1-senior-lpga-championship
[*] Retrieved 108811 bytes for http://www.espn.com/golf/story/_/id/24998354/he-called-shots-took-shots-now-johnny-miller-calls-career-retires-golf
[*] Retrieved 104684 bytes for http://www.espn.com/golf/story/_/id/24998446/johnny-miller-retire-nbc-golf-analyst-learned-how-team-player
[*] Retrieved 119802 bytes for http://www.

[*] Retrieved 16838 bytes for http://www.espn.com/golf/story/_/id/25463369/golf-presidents-cup-tiger-woods-1998-defeat-burns?device=featurephone
[*] Retrieved 14403 bytes for http://www.espn.com/golf/story/_/id/25477734/south-african-open-long-range-eagle-17th-gives-lead-schwartzel-south-african-open?device=featurephone
[*] Retrieved 12684 bytes for http://www.espn.com/golf/story/_/id/25479518/trio-teams-tied-qbe-shootout-lead-13-59?device=featurephone
[*] Retrieved 13298 bytes for http://www.espn.com/golf/story/_/id/25485380/gary-woodland-charley-hoffman-shoot-64-share-qbe-shootout-lead?device=featurephone
[*] Retrieved 119683 bytes for http://www.espn.com/golf/story/_/id/25490100/louis-oosthuizen-wins-south-african-open-charl-schwartzel-falls-away
[*] Retrieved 14011 bytes for http://www.espn.com/golf/story/_/id/25490100/louis-oosthuizen-wins-south-african-open-charl-schwartzel-falls-away?device=featurephone
[*] Retrieved 116264 bytes for http://www.espn.com/golf/story/_/id/25501049/

[*] Retrieved 122974 bytes for http://www.espn.com/golf/story/_/id/9301202/sergio-garcia-known-better
[*] Retrieved 122803 bytes for http://www.espn.com/golf/story/_/id/9438963/kenny-perry-claims-senior-players-first-major-title
[*] Retrieved 119280 bytes for http://www.espn.com/golf/story/_/id/9477785/jordan-spieth-wins-pga-tour-john-deere-classic
[*] Retrieved 119651 bytes for http://www.espn.com/golf/story/_/id/9485529/tiger-woods-signs-new-endorsement-contract-nike-agent-confirms
[*] Retrieved 115033 bytes for http://www.espn.com/golf/story/_/id/9537998/ben-martin-wins-webcom-tour-mylan-classic
[*] Retrieved 116217 bytes for http://www.espn.com/golf/story/_/id/9634499/jordan-spieth-20-picked-presidents-cup
[*] Retrieved 116896 bytes for http://www.espn.com/golf/story/_/id/9658547/wisconsin-golf-course-officials-threatened-9-11-promotion-backfires
[*] Retrieved 114289 bytes for http://www.espn.com/golf/story/_/id/9791478/world-golf-hall-fame-review-selection-process
[*] Retrieved 11

In [6]:
#2 change file name here based on sub-topic
file = open("./input/mr/cc_golf.txt", "w", encoding="utf-8") 
file.write(text)
file.close()